<a href="https://colab.research.google.com/github/koppacha/diffusers/blob/main/txt2img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Feb 28 10:31:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    32W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Diffusersをインストール

In [7]:
!pip install diffusers transformers ftfy accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached diffusers-0.13.1-py3-none-any.whl (716 kB)
  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached accelerate-0.16.0-py3-none-any.whl (199 kB)
  Using cached tokenizers-0.13.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)


In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached huggingface_hub-0.12.1-py3-none-any.whl (190 kB)


In [6]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


テキストから画像を生成する

In [8]:
# coding: utf-8
# make sure you're logged in with `huggingface-cli login`

# Apple Siliconでない場合はtorchもインストールする
# import torch

import datetime

from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

def main():
    # 現在日時を取得
    t_delta = datetime.timedelta(hours=9)
    jst = datetime.timezone(t_delta, 'JST')
    now = datetime.datetime.now(jst)

    # 🤗にあるモデルを指定 https://huggingface.co
    # モデルはDiffusersに対応している必要がある

    # model_id = "stabilityai/stable-diffusion-2-1-base"
    model_id = "hakurei/waifu-diffusion"

    # パイプライン本体　Apple Siliconではtorch_dtype=torch.float16を入れるとクラッシュする
    scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
    pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler)

    # Apple Siliconを使う設定
    # pipe = pipe.to("mps")

    # Apple Siliconを使わない場合
    pipe = pipe.to("cuda")

    # Recommended if your computer has < 64 GB of RAM
    pipe.enable_attention_slicing()

    # TODO: パーツごとに変数を用意して合体する

    # ひあドキュメントでプロンプトを指定
    prompt = """
    (((masterpiece))) super fine illustration, an extremely cute and beautiful girl, 
    highly detailed beautiful face and eyes, look at viewer, cowboy shot,
    beautiful hair, solo, dynamic angle, dynamic angle, School uniform
    """.replace('\n', '')

    # ネガティブプロンプトを指定
    n_prompt = """
    flat color, flat shading, nsfw, retro style, poor quality, bad face, 
    bad fingers, bad anatomy, missing fingers, low res, cropped, signature, 
    watermark, username, artist name, text
    """.replace('\n', '')

    # 生成する画像の数
    cnt = 3

    # NSFWフィルターを無効にする
    def null_safety(images, **kwargs):
        return images, False

    pipe.safety_checker = null_safety

    # First-time “warmup” pass (see explanation above)
    # 各種オプション
    _ = pipe(prompt, num_inference_steps=1)

    for i in range(cnt):
        # Results match those from the CPU device after the warmup pass.
        image = pipe(prompt=prompt, negative_prompt=n_prompt).images[0]

        # 保存するファイル名
        image.save('/content/drive/MyDrive/' + now.strftime('%Y%m%d%H%M%S') + '-' + str(i).zfill(2) + ".png")


main()

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

画像から画像を生成

In [9]:
import torch
import requests
import datetime
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionImg2ImgPipeline

save_path = '/content/drive/MyDrive/Colab Notebooks/diffusers/outputs/'

t_delta = datetime.timedelta(hours=9)
jst = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(jst)

# load the pipeline
device = "cuda"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to(
    device
)

# let's download an initial image
url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"

response = requests.get(url)
init_image = Image.open(BytesIO(response.content)).convert("RGB")
init_image.thumbnail((768, 768))

prompt = "A fantasy landscape, trending on artstation"

images = pipe(prompt=prompt, image=init_image, strength=0.75, guidance_scale=7.5).images

i = 0

images[0].save('/content/drive/MyDrive/' + now.strftime('%Y%m%d%H%M%S') + '-' + str(i).zfill(2) + ".png")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


  0%|          | 0/37 [00:00<?, ?it/s]